<a href="https://colab.research.google.com/github/misayuproject/1128kigen/blob/main/try1126.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ドライブの接続
from google.colab import drive
drive.mount('/content/drive')

11月26日

SHAPで特徴量の寄与率を調べて下20パーセントを減らした後のLightGBMは精度が落ちる。→採用せずに行くか。

特徴量の追加、

CatBoosterのとXGBoosterのパラメータをOptuna使って調べるとか？

In [ ]:
#　18秒

#コアデータ処理
import pandas as pd                         # データを表のように扱うライブラリ
import numpy as np                          # 数値計算を速くするライブラリ
from datetime import datetime               #日付と時刻を扱うための基本的なクラス

# テキスト処理と感情分析
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer       #このツールは、テキストで表現されている感情的なトーン（ポジティブ、ネガティブ、ニュートラル）を判断するのに役立ちます。
from textblob import TextBlob               # 感情分析などの機能を提供するテキスト処理ライブラリ

# 特徴量エンジニアリングと前処理:
from scipy.sparse import hstack             # スパース行列(多くのゼロを含むデータを効率的に格納する方法であるスパース行列を操作する)
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDFと呼ばれる手法を使用してテキストを数値ベクトルに変換します。これは、ドキュメント内の単語の重要性を表すのに役立ちます。
from sklearn.feature_extraction.text import CountVectorizer # テキストデータ内の単語の出現回数を単純にカウントすることによって数値ベクトルを作成
from sklearn.decomposition import TruncatedSVD  # 次元削減に使用され、データの特徴の数を減らします。
from sklearn.model_selection import train_test_split  # データを訓練用と検証用に分けるために使う。機械学習モデルの性能を評価するために不可欠
from sklearn.preprocessing import StandardScaler    # 数値特徴を平均0、分散1になるようにスケーリングすることで標準化します
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   #テゴリデータを数値形式にエンコードするために使用

import gensim.downloader as api             # 事前学習済みモデルをダウンロードするための api をインポート

#予測モデルに関するライブラリ
import lightgbm as lgb                      # 勾配ブースティングフレームワーク
import xgboost as xgb                       # もう1つの強力な勾配ブースティングライブラリ
!pip install catboost                       # もう1つの勾配ブースティングライブラリ
from catboost import CatBoostRegressor      # CatBoostRegressor は、特にその回帰モデル（連続値の予測）を指します
from sklearn.ensemble import RandomForestRegressor  # これは、分類タスクと回帰タスクの両方によく使用
from sklearn.model_selection import KFold   #データをk個のフォールド（グループ）に分割し、交差検証を行うためのクラス
"""
動作:
1.データセットをk個のフォールドに分割します。
2.k回の学習と評価を繰り返します。
    各回で、k-1個のフォールドを学習データ、残りの1個のフォールドを検証データとして使用します。
    学習データでモデルを学習し、検証データで性能を評価します。
3.k回の評価結果を平均して、モデルの最終的な性能を評価します。
"""


from sklearn.metrics import mean_squared_error # 平均二乗誤差 (MSE) を計算する関数
"""
計算式　MSE = (1/n) * Σ(yi - ŷi)^2
"""


import re   #正規表現モジュール re、正規表現は、テキスト内で特定のパターンを検索したり、操作したりするための強力なツールです。
"""
re の用途:
パターンマッチング: テキスト内で特定のパターン (例えば、メールアドレス、電話番号、日付) を見つける。
テキストの置換: テキスト内の特定のパターンを別のテキストに置き換える。
テキストの抽出: テキストから特定のパターンに一致する部分を抽出する。
データクリーニング: データセット内のテキストデータをクリーニングし、標準化する (例えば、不要な空白を削除する、大文字と小文字を統一する)。
"""

import warnings
warnings.simplefilter('ignore')  # 不要な警告を表示しない
"""
1.import warnings: Pythonに組み込まれているwarningsモジュールをインポートします。このモジュールは、警告メッセージを管理するための機能を提供します。警告メッセージは、エラーほど深刻ではありませんが、潜在的な問題や非推奨の機能の使用などを示すために使われます。
2.warnings.simplefilter('ignore'): この行が実際に警告メッセージを非表示にする処理を行います。
    warnings.simplefilter() は、警告メッセージに対するフィルターを設定する関数です。
    'ignore' という引数を指定することで、全ての警告メッセージを無視するように設定しています。つまり、警告メッセージはコンソールやログに出力されなくなります。
"""


In [31]:
#データ読み込みセクション　8秒ーーーーーーーーーーーーーーーー
#予測モデルを訓練するためのデータセット
train = pd.read_csv('/content/drive/MyDrive/data/train.csv', index_col=0)
# 予測モデルに推論（予測)させるデータセット
test = pd.read_csv('/content/drive/MyDrive/data/test.csv', index_col=0)

In [33]:
# テキストをクリーニングする関数　preprocess_cleanの定義　12秒
start_time = datetime.now()
print(f"Start: {start_time}")

def preprocess_clean(train, test, columns):
    def clean_text(text):
        if pd.isnull(text):
            return ''
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', '', text)
        return text

    for col in columns:
        if col in train.columns:
            train[col] = train[col].fillna('').map(clean_text)
        else:
            print(f"Warning: Column '{col}' not found in train DataFrame")

        if col in test.columns:
            test[col] = test[col].fillna('').map(clean_text)
        else:
            print(f"Warning: Column '{col}' not found in test DataFrame")

    return train, test

# 実行
review_columns = ['Positive_Review', 'Negative_Review']
train, test = preprocess_clean(train, test, review_columns)
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

# 処理時間を計算
end_time = datetime.now()
print(f"End  : {end_time}")
processing_time = end_time - start_time
print(f"ProcessingTime: {processing_time}")

Start: 2024-11-26 01:46:53.566325
train : (283366, 14) - End
test : (231845, 13) - End
End  : 2024-11-26 01:47:04.914438
ProcessingTime: 0:00:11.348113


In [ ]:
# Positive_ReviewとNegative_Reviewの内容をいろいろな手法で分析する.
# BOWN,N-gram,TF-IDFの実行、埋め込み表現　・・・

start_time = datetime.now()
print(f"Start: {start_time}")

def preprocess_reviews(train, test, columns):

    # Bag-of-Words特徴量の追加　add_bow_features関数
    """
    CountVectorizerを使用して、テキストデータを単語の出現回数に基づいた数値ベクトルに変換します。
    stop_words='english'で英語のストップワード（一般的な単語）を削除します。
    max_features=100で出現頻度が高い上位100単語のみを特徴量として使用します。
    生成されたBag-of-Words特徴量は、trainとtestデータフレームに新しいカラムとして追加されます。
    """
    def add_bow_features(train, test, column, max_features=10):
        # Bag of Words特徴量の追加
        bow = CountVectorizer(stop_words='english', max_features=10)    #max_features=50減らす
        # 既にクリーニング済みのカラムデータをそのまま使用
        train_bow = bow.fit_transform(train[column]).toarray()  # 変更箇所
        test_bow = bow.transform(test[column]).toarray()       # 変更箇所

        for i in range(train_bow.shape[1]):
            train[f'{column}_bow_{i}'] = train_bow[:, i]
            test[f'{column}_bow_{i}'] = test_bow[:, i]

        return train, test


    # テキストデータから N-gram 特徴量を抽出し、データフレームに追加する関数の定義
    """
    N-gram とは、テキスト中に出現する連続した単語の組み合わせのことです。例えば、「the quick brown fox」という文から、
        uni-gram (1-gram): the, quick, brown, fox
        bi-gram (2-gram): the quick, quick brown, brown fox
        tri-gram (3-gram): the quick brown, quick brown fox
    といった N-gram を抽出できます。これらの N-gram を特徴量として使用することで、機械学習モデルはテキストデータのより深い意味を理解できるようになります。
    CountVectorizer を使用し、uni-gram と bi-gram を抽出することで、単語の出現頻度だけでなく、単語の組み合わせも考慮した特徴量を作成している。
    この関数は、指定されたカラム (column) のテキストデータから N-gram 特徴量を抽出し、データフレームに追加します。
    ngram_range: N-gram の範囲を指定します。デフォルトは (1, 2) で、uni-gram と bi-gram を抽出します。
    max_features: 追加する特徴量の上限を設定します。デフォルトは 100 で、出現頻度が高い上位 100 個の N-gram のみを特徴量として使用します。
    """
    def add_ngram_features(df, column, ngram_range=(1, 2), max_features=50):  # max_features = 50

        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)  # max_features を設定
        # 既にクリーニング済みのカラムデータをそのまま使用
        ngram_features = vectorizer.fit_transform(df[column])  # 変更箇所

        # 特徴量名をデータフレームに追加
        ngram_feature_names = vectorizer.get_feature_names_out()  # 抽出された特徴量の名前を取得します。
        for i, feature_name in enumerate(ngram_feature_names):
            # 抽出された各 N-gram 特徴量を、新しいカラムとしてデータフレームに追加します。
            df[f'{column}_ngram_{feature_name}'] = ngram_features[:, i].toarray().ravel()

        return df


    # TF-IDF特徴量を追加する関数(定義)
    """
    TF-IDFベクトライザーの適用: TfidfVectorizerを使用して、テキストデータをTF-IDF値に基づいた数値ベクトルに変換します。

    max_features=1000で出現頻度が高い上位1000単語のみを特徴量として使用します。
    stop_words='english'で英語のストップワードを削除します。
    特徴量の次元削減: TruncatedSVDを使用して、TF-IDF特徴量の次元数を削減します。

    n_components=20で次元数を20に削減します。
    random_state=42で結果の再現性を確保します。
    特徴量をデータフレームに追加: 生成されたTF-IDF特徴量は、trainとtestデータフレームに新しいカラムとして追加されます。

    関数の適用: add_tfidf_features関数を'Positive_Review'と'Negative_Review'カラムに適用し、TF-IDF特徴量を追加します。
    """
    def add_tfidf_features(train, test, column, n_components=100, max_features=100):

        # TF-IDFベクトライザーの適用
        tfidf = TfidfVectorizer(max_features=max_features, stop_words='english')
        tfidf_train = tfidf.fit_transform(train[column])
        tfidf_test = tfidf.transform(test[column])

        # 特徴量の次元削減
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        svd_train = svd.fit_transform(tfidf_train)
        svd_test = svd.transform(tfidf_test)

        # 特徴量をデータフレームに追加
        for i in range(n_components):
            train[f'TFIDF_{column}_{i}'] = svd_train[:, i]
            test[f'TFIDF_{column}_{i}'] = svd_test[:, i]

        return train, test


    # 埋め込み表現をデータフレームに追加する関数定義
    def add_embedding_features(df, column, model_name='glove-wiki-gigaword-100', max_features=50): # max_features を引数に追加

        # glove-wiki-gigaword-100 という名前の事前学習済みモデル (GloVe) を読み込みます。
        # このモデルは、Wikipediaなどの大規模なテキストデータから単語の埋め込み表現を学習しています。
        model = api.load(model_name)  # 事前学習済みモデルの読み込み

        # テキストを受け取り、その埋め込み表現を返す関数を定義
        def get_embedding(text):
            words = text.split()    # テキストを単語に分割
            embeddings = [model[word] for word in words if word in model]   #各単語の埋め込み表現を事前学習済みモデルから取得

            if embeddings:          # 単語の埋め込み表現が存在する場合、それらを平均してテキスト全体の埋め込み表現を計算
                return np.mean(embeddings, axis=0)
            else:                   # 単語の埋め込み表現が存在しない場合、ゼロベクトルを返す
                return np.zeros(model.vector_size)

        # get_embedding 関数を df の指定された column ( 'Positive_Review' または 'Negative_Review' ) に適用し、
        # 埋め込み表現をデータフレームに変換する
        embedding_features = df[column].apply(get_embedding).apply(pd.Series)

        # 埋め込み表現の特徴量数を max_features で指定された数に制限します。
        embedding_features = embedding_features.iloc[:, :max_features] # 列数を制限

        # 埋め込み表現のカラムに名前を付けます (例: 'Positive_Review_embedding_0', 'Positive_Review_embedding_1', ...)。
        embedding_features.columns = [f'{column}_embedding_{i}' for i in range(embedding_features.shape[1])]

        # 埋め込み表現を元のデータフレームに結合します。
        df = pd.concat([df, embedding_features], axis=1)

        return df


    # 極性と主観性をデータフレームに追加する関数の定義 合計4カラム追加される。
    def add_polarity_features(df, column):

        # TextBlob を使って、指定された column ( 'Positive_Review' または 'Negative_Review' ) のテキストの極性を計算し、{column}_polarity という名前の新しいカラムに追加します。
        df[f'{column}_polarity'] = df[column].apply(lambda x: TextBlob(column).sentiment.polarity)

        # 同様に、主観性を計算し、{column}_subjectivity という名前の新しいカラムに追加します。
        df[f'{column}_subjectivity'] = df[column].apply(lambda x: TextBlob(column).sentiment.subjectivity)

        return df



    # 5種の関数を呼び出す ['Positive_Review', 'Negative_Review']の二回
    """
    add_ngram_features 関数: データフレーム df に直接カラムを追加するため、return df がなくても元のデータフレームに反映される。
    add_embedding_features 関数: 新しいデータフレームを作成して df に代入するため、return df で更新された df を返す必要がある。
    """
    for col in columns:

        # add_bow_features関数実行
        train, test = add_bow_features(train, test, col, max_features=100)
        print(f"add_bow_features  : {datetime.now()} - End ")
        print(f"train : {train.shape} - End")
        print(f"test : {test.shape} - End")

        # N-gram 特徴量を抽出関数実行
        train = add_ngram_features(train, col, max_features=100)    # 使用例: 上位 〇〇 個の Ngram 特徴量のみを追加
        test = add_ngram_features(test, col, max_features=100)     # 使用例: 上位 〇〇 個の Ngram 特徴量のみを追加
        print(f"add_ngram_features : {datetime.now()} - End")
        print(f"train : {train.shape} - End")
        print(f"test : {test.shape} - End")

        # TF-IDF特徴量追加関数実行 +200
        train, test = add_tfidf_features(train, test, col, max_features=100)
        print(f"add_tfidf_features : {datetime.now()} - End")
        print(f"train : {train.shape} - End")
        print(f"test : {test.shape} - End")

        # 埋め込み表現追加関数実行 +200
        train = add_embedding_features(train, col, max_features=100)
        test = add_embedding_features(test, col, max_features=100)
        print(f"add_embedding_features : {datetime.now()} - End")
        print(f"train : {train.shape} - End")
        print(f"test : {test.shape} - End")

        # 極性と主観性を追加する関数実行 +4
        train = add_polarity_features(train, col)
        test = add_polarity_features(test, col)
        print(f"add_polarity_features : {datetime.now()} - End")
        print(f"train : {train.shape} - End")
        print(f"test : {test.shape} - End")

        print(f"{col} : {datetime.now()} - End")

    # preprocess_reviews関数終わり
    return train, test

# preprocess_reviews関数の実行
review_columns = ['Positive_Review', 'Negative_Review']
train, test = preprocess_reviews(train, test, review_columns)


# キーワードを抽出し、特徴量として追加する関数　⁺100
def preprocess_reviews2(df, max_features=100):
    def extract_keywords(text):
        words = text.lower().split()
        common_words = ['the', 'and', 'is', 'in', 'it', 'to', 'was', 'i', 'of', 'a', 'this']
        keywords = [word for word in words if word not in common_words]
        return ' '.join(keywords)

    df['Positive_Review_Keywords'] = df['Positive_Review'].apply(extract_keywords)
    df['Negative_Review_Keywords'] = df['Negative_Review'].apply(extract_keywords)

    # CountVectorizerを使ってキーワードをエンコード（フィーチャースペースを制限）
    vectorizer = CountVectorizer(max_features=max_features)

    positive_keywords_matrix = vectorizer.fit_transform(df['Positive_Review_Keywords'])
    negative_keywords_matrix = vectorizer.fit_transform(df['Negative_Review_Keywords'])

    # スパース行列を結合
    keywords_matrix = hstack([positive_keywords_matrix, negative_keywords_matrix])

    # スパース行列をデータフレームに変換する
    keywords_df = pd.DataFrame(keywords_matrix.toarray(), columns=[f'Keyword_{i}' for i in range(keywords_matrix.shape[1])])

    # 元のデータフレームに結合
    df = pd.concat([df.reset_index(drop=True), keywords_df.reset_index(drop=True)], axis=1)

    return df

# キーワードを抽出し、特徴量として追加する関数の実行
train = preprocess_reviews2(train, max_features=100)
test = preprocess_reviews2(test, max_features=100)
print(f"preprocess_reviews2 : {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")


# 処理時間を計算
end_time = datetime.now()
print(f"End  : {end_time}")
processing_time = end_time - start_time
print(f"ProcessingTime: {processing_time}")


Start: 2024-11-26 01:47:15.436099
add_bow_features  : 2024-11-26 01:47:21.219333 - End 
train : (283366, 24) - End
test : (231845, 23) - End
add_ngram_features : 2024-11-26 01:47:40.073792 - End
train : (283366, 124) - End
test : (231845, 123) - End
add_tfidf_features : 2024-11-26 01:47:51.717406 - End
train : (283366, 224) - End
test : (231845, 223) - End
add_embedding_features : 2024-11-26 01:49:38.101329 - End
train : (283366, 324) - End
test : (231845, 323) - End
add_polarity_features : 2024-11-26 01:51:10.248086 - End
train : (283366, 326) - End
test : (231845, 325) - End
Positive_Review : 2024-11-26 01:51:10.248225 - End
add_bow_features  : 2024-11-26 01:51:16.192020 - End 
train : (283366, 336) - End
test : (231845, 335) - End
add_ngram_features : 2024-11-26 01:51:36.725913 - End
train : (283366, 436) - End
test : (231845, 435) - End


In [ ]:
#Review項目以外の特徴量追加　#3、#4，#5
#レビュー日付、ホテル名、緯度経度といった情報から新しい特徴量を生成
start_time = datetime.now()
print(f"Start: {start_time}")

# 3. レビュー日付からの特徴量生成(定義)g
def extract_date_features(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column])
    df['review_weekday'] = df[date_column].dt.weekday   #レビューが投稿された曜日 (0:月曜日, 1:火曜日, ..., 6:日曜日)
    df['review_month'] = df[date_column].dt.month       #レビューが投稿された月 (1:1月, 2:2月, ..., 12:12月)
    df['review_quarter'] = df[date_column].dt.quarter   #レビューが投稿された四半期 (1:1-3月, 2:4-6月, 3:7-9月, 4:10-12月)
    df['review_season'] = df[date_column].dt.month % 12 // 3 + 1    #レビューが投稿された季節 (1:春, 2:夏, 3:秋, 4:冬)

    return df

# 3．レビュー日付からの特徴量生成(実行)
date_column = 'Review_Date'
train = extract_date_features(train, date_column)
test = extract_date_features(test, date_column)
print(f"extract_date_features End:  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

# 4. ホテル名の処理(定義)
def preprocess_hotel_names(train, test, column):
    train[f'{column}_length'] = train[column].apply(len)    #Hotel_Name_length: ホテル名の文字数
    test[f'{column}_length'] = test[column].apply(len)      #Hotel_Name_length: ホテル名の文字数

    train[f'{column}_frequency'] = train[column].map(train[column].value_counts())  #Hotel_Name_frequency: データセット全体におけるそのホテル名の出現頻度
    test[f'{column}_frequency'] = test[column].map(train[column].value_counts())    #Hotel_Name_frequency: データセット全体におけるそのホテル名の出現頻度

    return train, test

# 4．ホテル名の処理(実行)
hotel_name_column = 'Hotel_Name'
train, test = preprocess_hotel_names(train, test, hotel_name_column)
print(f"preprocess_hotel_names End:  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

# 5. 緯度・経度の組み合わせ(定義)
#機械学習モデルがホテルの位置情報をより効果的に利用できるようにするために設計
#train、testに、lat_lng と lat_lng_encoded という2つの新しいカラムを追加する
"""
1.緯度経度の結合:
lat カラム（緯度）と lng カラム（経度）の値を文字列に変換し、"_" で連結して新しい lat_lng カラムを作成します。
例えば、緯度が35.6895、経度が139.6917の場合、lat_lng カラムの値は "35.6895_139.6917" となります。
これにより、緯度と経度を組み合わせた単一の特徴量が生成されます。
2.ラベルエンコーディング:
LabelEncoder を使用して、lat_lng カラムの値を数値に変換します。
LabelEncoder は、カテゴリカルデータを数値に変換する際に使用される手法です。
fit_transform メソッドを使用して、train データフレームの lat_lng カラムに含まれるユニークな値にそれぞれ数値を割り当てます。
transform メソッドを使用して、test データフレームの lat_lng カラムの値を、train データフレームで割り当てられた数値に変換します。
これにより、lat_lng_encoded という新しいカラムが作成され、緯度経度の組み合わせが数値で表現されます。
"""
def preprocess_lat_lng(train, test, lat_column, lng_column):
    # K-meansクラスタリングの代わりに、簡単な組み合わせ特徴量を作成
    train['lat_lng'] = train[lat_column].astype(str) + '_' + train[lng_column].astype(str)
    test['lat_lng'] = test[lat_column].astype(str) + '_' + test[lng_column].astype(str)

    encoder = LabelEncoder()
    train['lat_lng_encoded'] = encoder.fit_transform(train['lat_lng'])
    test['lat_lng_encoded'] = encoder.transform(test['lat_lng'])

    return train, test

# 5．緯度・経度の組み合わせ(実行)
lat_column = 'lat'
lng_column = 'lng'
train, test = preprocess_lat_lng(train, test, lat_column, lng_column)
print(f"preprocess_lat_lng End:  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")



#一番古いコーディング部分
def preprocess_data(df):

    # レビューの文字数を計算する
    df['Positive_Review_Length'] = df['Positive_Review'].apply(len)
    df['Negative_Review_Length'] = df['Negative_Review'].apply(len)

    # レビューのポジティブ・ネガティブ比率
    df['Review_Word_Ratio'] = df['Positive_Review_Length'] / (df['Negative_Review_Length'] + 1)

    # ポジティブレビューの単語数を計算
    df['Positive_Review_Word_Count'] = df['Positive_Review'].apply(lambda x: len(x.split()))
    # ネガティブレビューの単語数を計算
    df['Negative_Review_Word_Count'] = df['Negative_Review'].apply(lambda x: len(x.split()))

    # VADERを使ったセンチメント分析
    analyzer = SentimentIntensityAnalyzer()
    df['Positive_Review_Sentiment'] = df['Positive_Review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    df['Negative_Review_Sentiment'] = df['Negative_Review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

    # 過去のレビュー数と平均スコア
    df['Reviews_Per_Hotel'] = df.groupby('Hotel_Name')['Total_Number_of_Reviews'].transform('sum')
    df['Average_Score_Per_Hotel'] = df.groupby('Hotel_Name')['Average_Score'].transform('mean')

    # ホテルのレビュー数の変化
    df['Review_Trend'] = df.groupby('Hotel_Name')['Total_Number_of_Reviews'].transform(lambda x: x.diff().fillna(0))
    df['Average_Score_Trend'] = df.groupby('Hotel_Name')['Average_Score'].transform(lambda x: x.diff().fillna(0))

    # レビュアーのアクティブ度
    df['Reviewer_Activity'] = df['Total_Number_of_Reviews_Reviewer_Has_Given'] / df['days_since_review'].apply(lambda x: int(x.split()[0]))

    # 月ごとの宿泊者満足度
    if 'Reviewer_Score' in df.columns:
        df['Monthly_Avg_Score'] = df.groupby('review_month')['Reviewer_Score'].transform('mean')

    # 否定的・肯定的なレビューの有無
    df['Has_Negative_Review'] = df['Negative_Review'].apply(lambda x: 1 if x.strip() else 0)
    df['Has_Positive_Review'] = df['Positive_Review'].apply(lambda x: 1 if x.strip() else 0)

    # レビュアーの詳細情報解析
    df['Trip_Type'] = df['Tags'].apply(lambda x: 1 if 'Leisure trip' in x else 0)
    df['With_Family'] = df['Tags'].apply(lambda x: 1 if 'Family with young children' in x or 'Family with older children' in x else 0)
    df['With_Partner'] = df['Tags'].apply(lambda x: 1 if 'Couple' in x else 0)
    df['Solo_Traveler'] = df['Tags'].apply(lambda x: 1 if 'Solo traveler' in x else 0)

    return df

#一番古いコーディング部分の実行
train = preprocess_data(train)
test = preprocess_data(test)
print(f"preprocess_data End:  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

#11月25日特徴量追加

#時間特徴量の生成
# レビューの時間帯（朝、昼、夜）を特徴量として追加
def add_time_features(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column])
    df['review_hour'] = df[date_column].dt.hour
    df['review_time_of_day'] = df['review_hour'].apply(lambda x: 'morning' if 5 <= x < 12 else ('afternoon' if 12 <= x < 17 else 'evening'))
    return df
# 実行
date_column = 'Review_Date'
train = add_time_features(train, date_column)
test = add_time_features(test, date_column)
print(f"add_time_features :  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

# テキストの読みやすさ指標
!pip install textstat
from textstat import textstat
# テキストの読みやすさ指標を追加
def add_readability_features(df, columns):
    for col in columns:
        df[f'{col}_flesch_reading_ease'] = df[col].apply(lambda x: textstat.flesch_reading_ease(x) if pd.notnull(x) else 0)
    return df
# 実行
review_columns = ['Positive_Review', 'Negative_Review']
train = add_readability_features(train, review_columns)
test = add_readability_features(test, review_columns)
print(f"add_readalility_features:  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

#クラスタリングによる特徴量
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# テキストデータを数値ベクトルに変換してからクラスタリングを行います
def add_clustering_features(df, columns, n_clusters=5):
    for col in columns:
        # TF-IDFベクトル化
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(df[col].fillna(''))

        # KMeansクラスタリング
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        df[f'{col}_cluster'] = kmeans.fit_predict(tfidf_matrix)
    return df
# 実行
review_columns = ['Positive_Review', 'Negative_Review']
train = add_clustering_features(train, review_columns)
test = add_clustering_features(test, review_columns)
print(f"add_clustering_features :  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

"""
#エンティティ認識　→時間かかる
import spacy

# SpaCyを使用してエンティティを抽出し、特徴量として追加
nlp = spacy.load("en_core_web_sm")

def add_entity_features(df, columns):
    for col in columns:
        df[f'{col}_entities'] = df[col].apply(lambda x: ' '.join([ent.label_ for ent in nlp(x).ents]) if pd.notnull(x) else '')
    return df
# 実行
review_columns = ['Positive_Review', 'Negative_Review']
train = add_entity_features(train, review_columns)
test = add_entity_features(test, review_columns)
print(f"add_entity features :  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")
"""

# 既存の特徴量の積や商などの相互作用項を生成
def add_interaction_features(df, columns):
    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col1 = columns[i]
            col2 = columns[j]
            df[f'{col1}_x_{col2}'] = df[col1] * df[col2]
            df[f'{col1}_div_{col2}'] = df[col1] / (df[col2] + 1e-5)  # ゼロ除算を防ぐために小さな値を追加
    return df
# 実行
numeric_columns = ['review_hour', 'review_weekday', 'review_month', 'review_quarter', 'review_season']
train = add_interaction_features(train, numeric_columns)
test = add_interaction_features(test, numeric_columns)
print(f"add_interaction_features : {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")



#4. タグの埋め込み表現
from gensim.models import Word2Vec
def add_tag_embeddings(df, tag_column, vector_size=50):
    tags = df[tag_column].dropna().apply(lambda x: x.split(',')).tolist()
    model = Word2Vec(tags, vector_size=vector_size, window=5, min_count=1, sg=1)
    def get_tag_vector(tags):
        tag_vecs = [model.wv[tag] for tag in tags if tag in model.wv]
        if tag_vecs:
            return np.mean(tag_vecs, axis=0)
        else:
            return np.zeros(vector_size)
    tag_vectors = df[tag_column].apply(lambda x: get_tag_vector(x.split(',')) if pd.notnull(x) else np.zeros(vector_size))
    tag_vectors_df = pd.DataFrame(tag_vectors.tolist(), index=df.index)
    return pd.concat([df, tag_vectors_df], axis=1)
#実行
train = add_tag_embeddings(train, 'Tags')
test = add_tag_embeddings(test, 'Tags')
print(f"add_tag_embeddings :  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

#タグのクラスタリング
from sklearn.cluster import KMeans
def add_tag_clusters(df, tag_column, n_clusters=5):
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
    tag_matrix = vectorizer.fit_transform(df[tag_column].fillna(''))
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(tag_matrix)
    df[tag_column + '_cluster'] = kmeans.labels_
    return df
#実行
train = add_tag_clusters(train, 'Tags')
test = add_tag_clusters(test, 'Tags')
print(f"add_tag_clusters :  {datetime.now()} - End")
print(f"train : {train.shape} - End")
print(f"test : {test.shape} - End")

# 処理時間を計算
end_time = datetime.now()
print(f"End  : {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")


In [ ]:
#6まで全体のバックアップ取得
train_bkup6 = train.copy()
test_bkup6 = test.copy()

print(train_bkup6.shape)
print(test_bkup6.shape)

#(283366, 327)
#(231845, 325)

In [ ]:
#バックアップから戻す
train = train_bkup6.copy()
test = test_bkup6.copy()

print(train.shape)
print(test.shape)


In [ ]:
print(train.info())
print(test.info())


In [ ]:
#LightGBM単品コード書き直し　11月26日

start_time = datetime.now()
print(f"Start: {start_time}")

# 訓練用データセットの数値型以外の列を削除する
train = train.select_dtypes(include=['int64', 'float64', 'int32'])

# 推論用データセットにもレビュースコア以外の同様の列を残す
test = test[[col for col in train.columns if col in test.columns]]

# 訓練用データセットからターゲットを分離する
X = train.drop('Reviewer_Score', axis=1)
y = train['Reviewer_Score']

# テストデータに存在しない列を追加して0で埋める
missing_cols = set(X.columns) - set(test.columns)
for col in missing_cols:
    test[col] = 0

# 列の順番を訓練データと同じにする
test = test[X.columns]

# 無限大の値を置換
X.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

# 欠損値を平均値で補完 (他の方法でも補完可能)
X = X.fillna(X.mean())            #平均値で埋める
test = test.fillna(test.mean())
# 例：X_train = X_train.dropna()  # NaN値を含む行を削除

# float64に型変換
X = X.astype(np.float64)
test = test.astype(np.float64)

# KFoldによる分割設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 結果を保存するリスト
rmses = []
test_preds_lgb = np.zeros(test.shape[0])

# 各モデルの最適なハイパーパラメータの設定
lgb_params = {
    'learning_rate': 0.0992812083091482,   #0.05から変更
    'min_child_samples': 61,    #
    'n_estimators': 1000,   #500から変更
    'num_leaves': 250,
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42
}

# 各foldでの訓練と評価
for fold, (train_index, valid_index) in enumerate(kf.split(X)):
    print(f"Fold {fold+1} Start:{datetime.now()}")

    # 訓練用データと検証用データに分割
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]


    # LightGBMモデルの訓練
    lgb_model = lgb.train(
        lgb_params,
        lgb.Dataset(X_train, label=y_train),
        valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    print(f"Fold {fold+1} LightGBM time:{datetime.now()}")


    # 検証用データセットに対する予測
    y_pred_lgb = lgb_model.predict(X_valid, num_iteration=lgb_model.best_iteration)

    # モデルの性能をRMSEで評価
    rmse_lgb = mean_squared_error(y_valid, y_pred_lgb, squared=False)
    print(f"Fold {fold+1} RMSE (LightGBM): {rmse_lgb:.4f}")

    # 各foldの結果を保存
    rmses.append((rmse_lgb,))    # 単一要素のタプルとして追加

    # テストデータに対する予測を平均
    test_preds_lgb += lgb_model.predict(test, num_iteration=lgb_model.best_iteration) / kf.n_splits

# 全体の平均RMSE
mean_rmse_lgb = sum([rmse[0] for rmse in rmses]) / len(rmses)

print(f"Mean RMSE (LightGBM): {mean_rmse_lgb:.4f}")

# アンサンブル予測の平均
final_test_preds = (test_preds_lgb) / 1

# 予測結果を保存する
submit = pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv', header=None)
submit[1] = final_test_preds
submit.to_csv('/content/drive/MyDrive/data/submit_LightGBM1.csv', header=None, index=False)

submit.head()

# 処理時間を計算
end_time = datetime.now()
print(f"End: {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")



In [ ]:
import shap
import lightgbm as lgb

# ... (LightGBM モデルの学習) ...

# SHAP の Explainer オブジェクトを作成
explainer = shap.TreeExplainer(lgb_model)

# SHAP 値を計算
shap_values = explainer.shap_values(test_selected) # X_test は説明したいデータ

# SHAP の summary plot を表示
shap.summary_plot(shap_values, test_selected)

In [ ]:
# 特徴量重要度を取得
feature_importance = lgb_model.feature_importance()  # 既存のモデルから重要度を取得
feature_importance_df = pd.DataFrame({'feature': lgb_model.feature_name(), 'importance': feature_importance})

# 重要度でソート
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# ランキングを表示
print(feature_importance_df)  # DataFrame全体を表示

# 上位N件のみ表示したい場合 (例: 上位50件)
N = 200
print(feature_importance_df.head(N))

# 重要度の低い特徴量を削除 (例: 重要度が下位20%の特徴量を削除)
threshold = feature_importance_df['importance'].quantile(0.2)
selected_features = feature_importance_df[feature_importance_df['importance'] > threshold]['feature'].tolist()

# selected_features に含まれるカラムが X に存在するか確認
# 存在しないカラムがあれば警告を表示し、selected_features から削除
missing_features = [f for f in selected_features if f not in X.columns]
if missing_features:
    print(f"警告: 以下の特徴量は X に存在しません: {missing_features}")
    selected_features = [f for f in selected_features if f in X.columns]




# 選択された特徴量のみでデータを準備
X_selected = X[selected_features]  # Xは特徴量データ
test_selected = test[selected_features]

# test_selected に不足している列を取得する
missing_cols = set(X_selected.columns) - set(test_selected.columns)

# test_selected に不足している列を追加し、0 で埋める
for col in missing_cols:
    test[col] = 0

# 列の順序が同じであることを確認する
test_selected = test_selected[X_selected.columns]


# KFoldによる分割設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 結果を保存するリスト
rmses = []
test_preds = np.zeros(test_selected.shape[0])

# 各foldでの訓練と評価(2回目)
for fold, (train_index, valid_index) in enumerate(kf.split(X)):
    print(f"2回目のFold {fold+1} Start:{datetime.now()}")

    # 訓練用データと検証用データに分割 (選択された特徴量を使用)
    X_train, X_valid = X_selected.iloc[train_index], X_selected.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]  # yはターゲット変数

    # LightGBM用のデータセットに変換 (選択された特徴量を使用)
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)


    # LightGBMモデルの訓練
    lgb_model = lgb.train(
        lgb_params,
        lgb.Dataset(X_train, label=y_train),
        valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    print(f"2回目のFold {fold+1} LightGBM time:{datetime.now()}")


    # 検証用データセットに対する予測
    y_pred_lgb = lgb_model.predict(X_valid, num_iteration=lgb_model.best_iteration)

    # モデルの性能をRMSEで評価
    rmse_lgb = mean_squared_error(y_valid, y_pred_lgb, squared=False)
    print(f"2回目のFold {fold+1} RMSE (LightGBM): {rmse_lgb:.4f}")

    # 各foldの結果を保存
    rmses.append((rmse_lgb,))    # 単一要素のタプルとして追加

    # テストデータに対する予測を平均
    test_preds_lgb += lgb_model.predict(test_selected, num_iteration=lgb_model.best_iteration) / kf.n_splits

# 全体の平均RMSE
mean_rmse_lgb = sum([rmse[0] for rmse in rmses]) / len(rmses)

print(f"2回目のMean RMSE (LightGBM): {mean_rmse_lgb:.4f}")

# アンサンブル予測の平均
final_test_preds = (test_preds_lgb) / 1

# 予測結果を保存する
submit = pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv', header=None)
submit[1] = final_test_preds
submit.to_csv('/content/drive/MyDrive/data/submit_LightGBM2.csv', header=None, index=False)
feature_importance_df.to_csv('/content/drive/MyDrive/data/feature_importance.csv', index=False)
submit.head()




In [ ]:
feature_importance_df.to_csv('/content/drive/MyDrive/data/feature_importance.csv', index=False)


In [ ]:
#アンサンブルコード書き直し　11月25日
#ランダムフォレスト除外
#3つのモデルのハイパーパラメータ変更
"""
from datetime import datetime
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
"""
# 必要なモジュールのインポートを追加→結局使わない
from xgboost.callback import EarlyStopping

start_time = datetime.now()
print(f"Start: {start_time}")

# 訓練用データセットの数値型以外の列を削除する
train = train.select_dtypes(include=['int64', 'float64', 'int32'])

# 推論用データセットにもレビュースコア以外の同様の列を残す
test = test[[col for col in train.columns if col in test.columns]]

# 訓練用データセットからターゲットを分離する
X = train.drop('Reviewer_Score', axis=1)
y = train['Reviewer_Score']

# テストデータに存在しない列を追加して0で埋める
missing_cols = set(X.columns) - set(test.columns)
for col in missing_cols:
    test[col] = 0


# 列の順番を訓練データと同じにする
test = test[X.columns]

# 無限大の値を置換
X.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

# 欠損値を平均値で補完 (他の方法でも補完可能)
X = X.fillna(X.mean())            #平均値で埋める
test = test.fillna(test.mean())
# 例：X_train = X_train.dropna()  # NaN値を含む行を削除

# float64に型変換
X = X.astype(np.float64)
test = test.astype(np.float64)


# KFoldによる分割設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 結果を保存するリスト
rmses = []
test_preds_lgb = np.zeros(test.shape[0])
test_preds_rf = np.zeros(test.shape[0])
test_preds_xgb = np.zeros(test.shape[0])
test_preds_cat = np.zeros(test.shape[0])

# 各モデルの最適なハイパーパラメータの設定
lgb_params = {
    'learning_rate': 0.0992812083091482,   #0.05から変更
    'min_child_samples': 61,    #
    'n_estimators': 1000,   #500から変更
    'num_leaves': 250,
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42
}
#Optunaで算出したパラメータ
#Best parameters: {'learning_rate': 0.0992812083091482, 'num_leaves': 250, 'min_child_samples': 61}
#Best RMSE: 1.012224322083124
"""
rf_params = {
    'n_estimators': 25, #ランダムフォレストを構成する決定木の数,100〜200 程度から始め、交差検証などで最適な値を探索するのが良い
    'random_state': 42, #乱数シード
    'max_depth': 10,    #各決定木の最大深さ None（制限なし）から始め、交差検証などで最適な値を探索するのが良い
    'min_samples_split': 2, #ノードを分割するために必要な最小サンプル数 2〜10 程度から始め、交差検証などで最適な値を探索するのが良い
    'min_samples_leaf': 1,  #葉ノードに存在する最小サンプル数 1〜5 程度から始め、交差検証などで最適な値を探索するのが良い
    #'max_features': 'auto',    #各決定木でノードを分割する際に考慮する特徴量の最大数 'auto'（特徴量の数の平方根）、'sqrt'、'log2' など
    'n_jobs': -1    #並列処理に使用するコア数  -1 を指定すると、すべてのコアを使用
}
"""
xgb_params = {
    'learning_rate': 0.1,  #デフォルトは0.3　一般的に、0.01〜0.1程度の値が使用される。
    'n_estimators': 1000,    #ブースター（決定木）の数
    'objective': 'reg:squarederror',    #reg:squarederror（回帰）
    'random_state': 42
}

cat_params = {
    'iterations': 500,      #学習の反復回数 (決定木の数)
    'learning_rate': 0.1,  #学習率
    'depth': 6,             #決定木の最大深度
    'loss_function': 'RMSE',#損失関数
    'random_seed': 42,      #乱数シード
    'verbose': 0            #学習過程の出力レベル　0表示なし　1表示あり
}

# 各foldでの訓練と評価
for fold, (train_index, valid_index) in enumerate(kf.split(X)):
    print(f"Fold {fold+1} Start:{datetime.now()}")

    # 訓練用データと検証用データに分割
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]


    # LightGBMモデルの訓練
    lgb_model = lgb.train(
        lgb_params,
        lgb.Dataset(X_train, label=y_train),
        valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    print(f"Fold {fold+1} LightGBM time:{datetime.now()}")


    # XGBoostモデルの訓練
    xgb_model = xgb.XGBRegressor(**xgb_params)
    early_stop = EarlyStopping(rounds=50) # EarlyStoppingコールバックを作成
    #xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[early_stop], verbose=False) # callbacks引数に追加
    xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    print(f"Fold {fold+1} XGBoost time:{datetime.now()}")

    # CatBoostモデルの訓練
    cat_model = CatBoostRegressor(**cat_params)
    cat_model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=False)
    print(f"Fold {fold+1} CatBoost time:{datetime.now()}")

    # ランダムフォレストモデルの訓練
    #rf_model = RandomForestRegressor(**rf_params)
    #rf_model.fit(X_train, y_train)
    #print(f"Fold {fold+1} ランダムフォレスト time:{datetime.now()}")


    # 検証用データセットに対する予測
    y_pred_lgb = lgb_model.predict(X_valid, num_iteration=lgb_model.best_iteration)
    y_pred_xgb = xgb_model.predict(X_valid)
    y_pred_cat = cat_model.predict(X_valid)
    #y_pred_rf = rf_model.predict(X_valid)

    # モデルの性能をRMSEで評価
    rmse_lgb = mean_squared_error(y_valid, y_pred_lgb, squared=False)
    rmse_xgb = mean_squared_error(y_valid, y_pred_xgb, squared=False)
    rmse_cat = mean_squared_error(y_valid, y_pred_cat, squared=False)
    #rmse_rf = mean_squared_error(y_valid, y_pred_rf, squared=False)
    print(f"Fold {fold+1} RMSE (LightGBM): {rmse_lgb:.4f}")
    print(f"Fold {fold+1} RMSE (XGBoost): {rmse_xgb:.4f}")
    print(f"Fold {fold+1} RMSE (CatBoost): {rmse_cat:.4f}")
    #print(f"Fold {fold+1} RMSE (RandomForest): {rmse_rf:.4f}")

    # 各foldの結果を保存
    #rmses.append((rmse_lgb, rmse_rf, rmse_xgb, rmse_cat))
    rmses.append((rmse_lgb, rmse_xgb, rmse_cat))

    # テストデータに対する予測を平均
    test_preds_lgb += lgb_model.predict(test, num_iteration=lgb_model.best_iteration) / kf.n_splits
    test_preds_xgb += xgb_model.predict(test) / kf.n_splits
    test_preds_cat += cat_model.predict(test) / kf.n_splits
    #test_preds_rf += rf_model.predict(test) / kf.n_splits

# 全体の平均RMSE
mean_rmse_lgb = sum([rmse[0] for rmse in rmses]) / len(rmses)
mean_rmse_xgb = sum([rmse[1] for rmse in rmses]) / len(rmses)
mean_rmse_cat = sum([rmse[2] for rmse in rmses]) / len(rmses)
#mean_rmse_rf = sum([rmse[3] for rmse in rmses]) / len(rmses)

print(f"Mean RMSE (LightGBM): {mean_rmse_lgb:.4f}")
print(f"Mean RMSE (XGBoost): {mean_rmse_xgb:.4f}")
print(f"Mean RMSE (CatBoost): {mean_rmse_cat:.4f}")
#print(f"Mean RMSE (RandomForest): {mean_rmse_rf:.4f}")

# アンサンブル予測の平均
#final_test_preds = (test_preds_lgb + test_preds_rf + test_preds_xgb + test_preds_cat) / 4
final_test_preds = (test_preds_lgb + test_preds_xgb + test_preds_cat) / 3

# 予測結果を保存する
submit = pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv', header=None)
submit[1] = final_test_preds
submit.to_csv('/content/drive/MyDrive/data/submit_ensemble.csv', header=None, index=False)

submit.head()

# 処理時間を計算
end_time = datetime.now()
print(f"End: {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")



In [ ]:
"""コピー用
start_time = datetime.now()
print(f"Start: {start_time}")

end_time = datetime.now()
print(f"End: {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")

以降は、チューニング用のモジュール
Optuna、

GridSearchCV

In [ ]:
!pip install optuna
import optuna

#learning_rate, num_leaves, max_depthはモデルの性能に大きく影響するため、優先的にチューニングすることをお勧めします。
def objective(trial):
    # ハイパーパラメータの範囲を指定
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        # ... other hyperparameters ...
    }

    # LightGBMモデルの学習
    lgb_model = lgb.train(
        param,
        lgb.Dataset(X_train, label=y_train),
        valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    # 予測とRMSEの計算
    y_pred = lgb_model.predict(X_valid, num_iteration=lgb_model.best_iteration)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)

    return rmse  # 最小化したい指標を返す

study = optuna.create_study(direction='minimize')  # 最小化したい場合は 'minimize'
study.optimize(objective, n_trials=100)  # 試行回数

# 最適なハイパーパラメータ
print(f"Best parameters: {study.best_params}")
print(f"Best RMSE: {study.best_value}")




In [ ]:
start_time = datetime.now()
print(f"Start xgb_model: {start_time}")

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        # ... other hyperparameters ...
    }

    xgb_model = xgb.XGBRegressor(**param)
    xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    y_pred = xgb_model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータ
print(f"Best parameters: {study.best_params}")
print(f"Best RMSE: {study.best_value}")



end_time = datetime.now()
print(f"End xgb_model: {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")



In [ ]:
start_time = datetime.now()
print(f"Start Cat: {start_time}")

def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        # ... other hyperparameters ...
    }

    cat_model = CatBoostRegressor(**param, verbose=False)
    cat_model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

    y_pred = cat_model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータ
print(f"Best parameters: {study.best_params}")
print(f"Best RMSE: {study.best_value}")


end_time = datetime.now()
print(f"End cat: {end_time}")
processing_time = end_time - start_time
print(f"ProcessTime: {processing_time}")


In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print("Best parameters: ", study.best_params)
print("Best MSE: ", study.best_value)

In [ ]:
from sklearn.model_selection import GridSearchCV

# ハイパーパラメータの候補を設定
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'min_child_samples': [20, 30, 50]
}

# LightGBMのモデルを設定
lgb_model = lgb.LGBMRegressor(objective='regression', random_state=42)

# GridSearchCVを使用して最適なハイパーパラメータを探索
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# 最適なハイパーパラメータを表示
print(f"Best parameters found: {grid_search.best_params_}")

LightGBMのパラメータチューニングには以下の主要なパラメータがあります。これらはモデルの性能に大きな影響を与えるため、優先して調整すると良いでしょう。

num_leaves: 決定木の最大葉数。大きくするとモデルの複雑さが増すが、過学習のリスクもある。

learning_rate: 学習率。小さくすると学習が安定するが、計算時間が増える。

n_estimators: 決定木の数。多くすると精度が上がるが、計算コストも増える。

max_depth: 決定木の最大深度。深くするとモデルの複雑さが増すが、過学習のリスクもある。

min_child_weight: 子ノードの最小サンプル数。小さくすると過学習しやすくなる。

subsample: バギングの割合。過学習を防ぐためにデータの一部を使用する。

colsample_bytree: 決定木ごとに使用する特徴量の割合。過学習を防ぐために特徴量の一部を使用する。

CatBoostのパラメータチューニングには、以下の主要なパラメータがあります。これらはモデルの性能に大きな影響を与えるため、優先して調整すると良いでしょう。

iterations: 学習回数。多くすると精度が上がるが、計算コストも増える。

learning_rate: 学習率。小さくすると学習が安定するが、計算時間が増える。

depth: 決定木の深さ。深くするとモデルの複雑さが増すが、過学習のリスクもある。

l2_leaf_reg: L2正則化項。大きくすると過学習を防ぐが、精度が下がる可能性がある。

bagging_temperature: バギングの温度。大きくするとデータの一部を使用する確率が高くなる。

border_count: 数値特徴量をカテゴリカルに変換する際の分割数。多くすると精度が上がるが、計算コストも増える。

random_strength: 決定木の分割時にランダム性を追加する強さ。過学習を防ぐために使用する。

これらのパラメータを調整することで、モデルの性能を最適化することができます。Optunaなどの自動化されたチューニングツールを使用して、効率的に最適なパラメータを見つけることをお勧めします。

XGBoostのパラメータチューニングには、以下の主要なパラメータがあります。これらはモデルの性能に大きな影響を与えるため、優先して調整すると良いでしょう。

eta (learning_rate): 学習率。小さくすると学習が安定するが、計算時間が増える。

max_depth: 決定木の深さ。深くするとモデルの複雑さが増すが、過学習のリスクもある。

min_child_weight: 葉の重みの最小値。値を大きくすると過学習を防ぐが、精度が下がる可能性がある。

subsample: 学習データのサンプリング率。値を小さくすると過学習を防ぐが、あまり小さくしすぎるとモデルの性能が低下する。

colsample_bytree: 決定木ごとの特徴量のサンプリング率。値を小さくすると過学習を防ぐが、あまり小さくしすぎるとモデルの性能が低下する。

gamma: ノード分割の基準となる損失減少の最小値。値を大きくすると過学習を防ぐが、精度が下がる可能性がある。

lambda (reg_lambda): L2正則化項。大きくすると過学習を防ぐが、精度が下がる可能性がある。

alpha (reg_alpha): L1正則化項。大きくすると過学習を防ぐが、精度が下がる可能性がある.

これらのパラメータを調整することで、モデルの性能を最適化することができます。Optunaなどの自動化されたチューニングツールを使用して、効率的に最適なパラメータを見つけることをお勧めします。